In [ ]:
import pandas as pd
import requests
import re
import numpy as np
import ast
import time
from datetime import date
today = date.today()

### We need to get the list of all hashes first

In [ ]:
with open('address.txt', 'r') as file:
    address = file.read()
    
address_to_get_all = address + '?platform=ethereum&limit=100&page='

data = pd.DataFrame(columns=['category', 
                             'categories', 
                             'created', 
                             'name', 
                             'rank', 
                             'slug', 
                             'teaser'])

In [ ]:
# we have the maximum of 20 pages, if the limit set to 100 apps per page

In [ ]:
for page in range(1, 21):
    test = requests.get(f'{address_to_get_all}{page}')
    for i, item in enumerate(test.json()['items']):
        category = test.json()['items'][i]['categories']
        if len(category)>1:
            categories = ' '.join(category)
            category = category[0]
        else:
            categories = 0
            category = category[0]
        created = test.json()['items'][i]['created']
        name = test.json()['items'][i]['name']
        rank = test.json()['items'][i]['rank']
        slug = test.json()['items'][i]['slug']
        teaser = test.json()['items'][i]['teaser']
        df_temp = pd.DataFrame(data={'category': category, 
                                    'categories': categories,
                                    'created': created,
                                    'name': name,
                                    'rank': rank,
                                    'slug': slug,
                                    'teaser': teaser}, index=[0])
        data = pd.concat([data, df_temp])
    time.sleep(0.25)

In [ ]:
data = data.drop('categories', axis=1)

In [ ]:
data.to_csv(f'data/dapps_names_{today}.csv', index=False)

### For each dapp, here or each slug, retrieve a list of hashes

In [ ]:
data = pd.read_csv(f'data/dapps_names_{today}.csv')

In [ ]:
address_for_dapp = f'{address}/'
data.reset_index(drop=True, inplace=True)
hashes = pd.DataFrame(columns=['slug', 'hash', 'description', 'license'])

In [ ]:
for i, slug in enumerate(data.slug):
    slug = slug
    test = requests.get(f'{address_for_dapp}{data.slug[i]}')
    c = test.json()['item']['contractsMainnet']
    if len(c)==0:
        description = test.json()['item']['description']
        contract_hash = 0
        license = test.json()['item']['license']
        df_temp = pd.DataFrame(data={'slug': slug,
                                    'hash': contract_hash,
                                    'description': description,
                                    'license': license}, index=[0])
    elif len(c)==1: 
        description = test.json()['item']['description']
        contract_hash = c[0]
        license = test.json()['item']['license']
        df_temp = pd.DataFrame(data={'slug': slug,
                                    'hash': contract_hash,
                                    'description': description,
                                     'license': license}, index=[0])
    elif len(c)> 1: 
        df_temp = pd.DataFrame(columns=['slug', 'hash', 'description', 'license'])
        for contract in c:
            description = test.json()['item']['description']
            contract_hash = contract
            license = test.json()['item']['license']
            df_t = pd.DataFrame(data={'slug': slug,
                                        'hash': contract_hash,
                                        'description': description,
                                        'license': license}, index=[0])
            df_temp = pd.concat([df_temp, df_t])
    hashes = pd.concat([hashes, df_temp])
    time.sleep(0.25)

In [ ]:
hashes = hashes[hashes['hash']!=0]

In [ ]:
hashes.reset_index(drop=True, inplace=True)
#hashes.to_csv(f'data/hashes_with_licenses_{today}.csv', index=False)

dapps_df = hashes.merge(data, how='left', left_on='slug', right_on='slug')
dapps_df.to_csv(f'data/dapps_with_licences_{today}.csv', index=False)